* [Yelp, Not Kaggle Data Source](https://www.yelp.com/dataset/download)
* [Data Dictionary](https://www.yelp.com/dataset/documentation/main)

* [x] Get data into csv to work in pandas
* [ ] Perform sentiment classification with Hugging Face model (2024 SOTA)
* [ ] Extract from text key words that resulted in sentiment class
* [ ] topic modeling?
* [ ] other advanced analytics for reviews using LLM/HuggingFace
* [ ] compare competition side-by-side (Japanese restaurants)
* [ ] get criteria for ranking/qualities in school based on aggregation of reviews

In [1]:
!pip install duckdb jupysql -q transformers --quiet 

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

import re
from transformers import pipeline

Deploy Flask apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


2024-05-01 04:11:49.842312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 04:11:49.842538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 04:11:50.035578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Get relevant data (business, review, user)
Data Ingestion credit: https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88

In [3]:
business_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}"
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsPriceRange2': '2', 'CoatCheck': 'False', 'RestaurantsTakeOut': 'False', 'RestaurantsDelivery': 'False', 'Caters': 'False', 'WiFi': 'u'no'', 'BusinessParking': '{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}', 'WheelchairAccessible': 'True', 'HappyHour': 'False', 'OutdoorSeating': 'False', 'HasTV': 'False', 'RestaurantsReservations': 'False', 'DogsAllowed': 'False', 'ByAppointmentOnly': 'False'}","Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', 'Wednesday': '8:0-22:0', 'Thursday': '8:0-22:0', 'Friday': '8:0-23:0', 'Saturday': '8:0-23:0', 'Sunday': '8:0-22:0'}"
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeating': 'False', 'BusinessAcceptsCreditCards': 'False', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BikeParking': 'True', 'RestaurantsPriceRange2': '1', 'RestaurantsTakeOut': 'True', 'ByAppointmentOnly': 'False', 'WiFi': 'u'free'', 'Alcohol': 'u'none'', 'Caters': 'True'}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', 'Wednesday': '7:0-20:0', 'Thursday': '7:0-20:0', 'Friday': '7:0-21:0', 'Saturday': '7:0-21:0', 'Sunday': '7:0-21:0'}"
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'WheelchairAccessible': 'True', 'RestaurantsTakeOut': 'True', 'BusinessParking': '{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}', 'BikeParking': 'True', 'GoodForKids': 'True', 'Caters': 'False'}","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '12:0-22:0', 'Saturday': '12:0-22:0', 'Sunday': '12:0-18:0'}"


# Clean biz file

In [4]:
# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1] # getting biz in operation only


drop_columns = ['hours','is_open']
df_b.drop(drop_columns, axis=1, inplace=True)
df_b.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services"


In [5]:
# Getting restaurant related businesses
# get task and prompt columns only
query = """
select *
from df_b
where lower(categories) like '%restaurants%'
"""
biz_rest =duckdb.query(query).df()
biz_rest.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'True', '1', 'True', 'False', 'u'free'', 'u'none'', 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries"
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,"{'key': ['BusinessParking', 'BusinessAcceptsCreditCards', 'RestaurantsAttire', 'OutdoorSeating', 'RestaurantsReservations', 'Caters', 'RestaurantsTakeOut', 'Alcohol', 'Ambience', 'GoodForKids', 'RestaurantsPriceRange2', 'ByAppointmentOnly', 'CoatCheck', 'DogsAllowed', 'RestaurantsTableService', 'RestaurantsGoodForGroups', 'RestaurantsDelivery', 'WiFi', 'WheelchairAccessible', 'HasTV', 'HappyHour', 'DriveThru', 'BikeParking'], 'value': ['None', 'True', 'u'casual'', 'True', 'False', 'False', 'True', 'u'none'', 'None', 'True', '1', 'False', 'False', 'False', 'False', 'True', 'True', 'u'no'', 'True', 'True', 'False', 'True', 'False']}","Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants"
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,"{'key': ['RestaurantsAttire', 'RestaurantsGoodForGroups', 'BusinessAcceptsCreditCards', 'OutdoorSeating', 'GoodForKids', 'Alcohol', 'BusinessParking', 'DogsAllowed', 'RestaurantsTableService', 'ByAppointmentOnly', 'WiFi', 'RestaurantsPriceRange2', 'RestaurantsReservations', 'HasTV', 'RestaurantsDelivery', 'CoatCheck', 'Caters', 'RestaurantsTakeOut', 'DriveThru', 'HappyHour', 'WheelchairAccessible'], 'value': [''casual'', 'False', 'True', 'True', 'True', 'u'none'', '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'False', 'False', 'False', 'u'no'', '1', 'False', 'True', 'True', 'False', 'False', 'True', 'True', 'False', 'True']}","Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food"
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"{'key': ['Alcohol', 'OutdoorSeating', 'RestaurantsReservations', 'Ambience', 'GoodForMeal', 'BusinessParking'], 'value': [''none'', 'None', 'False', '{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}', '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}', '{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}']}","Vietnamese, Food, Restaurants, Food Trucks"
4,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,"{'key': ['RestaurantsReservations', 'RestaurantsTakeOut', 'RestaurantsAttire', 'GoodForKids', 'BikeParking', 'RestaurantsPriceRange2', 'BusinessAcceptsCreditCards', 'GoodForMeal', 'RestaurantsGoodForGroups', 'OutdoorSeating', 'WiFi', 'Alcohol', 'RestaurantsDelivery', 'BusinessParking', 'HasTV', 'Ambience'], 'value': ['False', 'True', ''casual'', 'True', 'False', '1', 'True', '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}', 'True', 'False', 'u'no'', ''none'', 'True', '{'garage': None, 'street': False, 'validated': None, 'lot': True, 'valet': False}', 'True', '{'touristy': None, 'hipster': None, 'romantic': False, 'divey': None, 'intimate': None, 'trendy': None, 'upscale': None, 'classy': None, 'casual': True}']}","American (Traditional), Restaurants, Diners, Breakfast & Brunch"


In [6]:
biz_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34987 entries, 0 to 34986
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   34987 non-null  object 
 1   name          34987 non-null  object 
 2   address       34987 non-null  object 
 3   city          34987 non-null  object 
 4   state         34987 non-null  object 
 5   postal_code   34987 non-null  object 
 6   latitude      34987 non-null  float64
 7   longitude     34987 non-null  float64
 8   stars         34987 non-null  float64
 9   review_count  34987 non-null  int64  
 10  attributes    34547 non-null  object 
 11  categories    34987 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 3.2+ MB


# Bring in review data and retain businesses with reviews only

In [7]:
review_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json'

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [8]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner join with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(biz_rest, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

540141 out of 1,000,000 related reviews
549947 out of 1,000,000 related reviews
524365 out of 1,000,000 related reviews
529735 out of 1,000,000 related reviews
546958 out of 1,000,000 related reviews
544052 out of 1,000,000 related reviews
538572 out of 1,000,000 related reviews


In [9]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,review_id,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'True', '1', 'True', 'False', 'u'free'', 'u'none'', 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area.",2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'True', '1', 'True', 'False', 'u'free'', 'u'none'', 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town.",2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'True', '1', 'True', 'False', 'u'free'', 'u'none'', 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite.",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking', 'BikeParking', 'RestaurantsPriceRange2', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters'], 'value': ['False', 'False', 'False', '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'True', '1', 'True', 'False', 'u'free'', 'u'none'', 'True']}","Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future 

In [10]:
len(biz_rest), len(df) # length of df should be longer with reviews fanning out on businesses (multiple reviews)

(34987, 3773770)

In [11]:
# which restaurants have the most reviews
query = """
select name
, count(*) as total_reviews
from df
group by 1
order by 2 desc
limit 10
"""
duckdb.query(query).df()

,name,total_reviews
0,McDonald's,17309
1,Chipotle Mexican Grill,9438
2,First Watch,9076
3,Acme Oyster House,8461
4,Taco Bell,8320
5,Chick-fil-A,8315
6,Oceana Grill,7516
7,Buffalo Wild Wings,7063
8,Panera Bread,6906
9,Domino's Pizza,6335


In [12]:
df.to_csv('yelp_biz_rev.csv')

# Get user data
Might be useful to pivot by user to see how they perform reviews in future iteration

## Need to perform chunking with user data as well

In [13]:
user_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json'

size = 1000000
user = pd.read_json(user_json_path, lines=True,
                      dtype={'user_id':str,'name':str,
                             'review_count':int,'yelping_since':str,
                             'friends':object,'useful':int,
                             'funny':int,'cool':int,
                             'fans':int, 'elite':object,
                            'average_stars':float, 'compliment_hot':int,
                            'compliment_more':int, 'compliment_profile':int,
                            'compliment_cute':int, 'compliment_list':int,
                            'compliment_note':int, 'compliment_plain':int,
                            'compliment_cool':int, 'compliment_funny':int,
                            'compliment_writer':int, 'compliment_photos':int},
                      chunksize=size)

In [14]:
# Bring in business, review df
df = pd.read_csv('/kaggle/working/yelp_biz_rev.csv')

/tmp/ipykernel_18/3348335960.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/working/yelp_biz_rev.csv')


In [15]:
# There are multiple chunks to be read
chunk_list = []
for chunk_user in user:
    # Renaming column name to avoid conflict with business name
    chunk_user = chunk_user.rename(columns={'name': 'customer_name', 'review_count': 'customer_review_count'})
    # Inner join with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df, chunk_user, on='user_id', how='inner') # only keep users that have made restaurant review(s)
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

3084973 out of 1,000,000 related reviews
688790 out of 1,000,000 related reviews


In [16]:
df.head(1)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
list(df)

['Unnamed: 0',
 'business_id',
 'name',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars',
 'review_count',
 'attributes',
 'categories',
 'review_id',
 'user_id',
 'review_stars',
 'text',
 'date',
 'customer_name',
 'customer_review_count',
 'yelping_since',
 'useful',
 'funny',
 'cool',
 'elite',
 'friends',
 'fans',
 'average_stars',
 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

In [18]:
# df.to_csv('yelp_final.csv')

In [19]:
# df to work with for sentiment analysis

query = """
select name as business_name
, city
, state
, stars
, review_count
, attributes
, categories
, review_id
, user_id
, review_stars
, text
, date
, customer_review_count
from df
"""
df_sent = duckdb.query(query).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
# smaller df to test and apply text cleaning and experiment with different algo's

# query = """
# select name as business_name
# , city
# , state
# , stars
# , review_count
# , attributes
# , categories
# , review_id
# , user_id
# , review_stars
# , text
# , date
# , customer_review_count
# from df
# """

query = """
select review_id
, text
from df
"""

df_sent2 = duckdb.query(query).df()
df_sent2

,review_id,text
0,BXQcBN0iAi1lAUxibGLFzA,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area."
1,uduvUCvi9w3T2bSGivCfXg,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town."
2,a0vwPOqDXXZuJkbBW2356g,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite."
3,MKNp_CdR2k2202-c8GN5Dw,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future events! Staff was nice also and the cake was really good.
4,D1GisLDPe84Rrk_R4X2brQ,"HK-STYLE MILK TEA: FOUR STARS\n\nNot quite sure why there are two Saint Honore Pastries business listings on Yelp, but I have been checking in to this one.\n\nBoyfriend and I go by it all the time while in Chinatown, but it never crossed our mind to try it. I think we might have tried it once a year or two ago, but it wasn't memorable. I decided to try the HK-style milk tea not too long ago and was hooked. I think I went here almost every day one week to get the milk tea, only to realize that I should REALLY watch my sugar intake...it's sooo good! \n\nI personally don't think it's better than Zhong Gang's HK-style milk tea because they have their off days when a certain lady makes them (not enough ice, too much ice, etc.).\n\nOn a side note: I wanted to try the crispy roast pork bun, but accidentally said the wrong name in Chinese and got the roast pork bun. It was decent enough, although I will get to trying the crispy roast pork bun soon! \n\nI will be back~"
...,...,...
3773758,laomHshWlsT0OXHGFB94iw,"Philly's best kept secret! Best nachos, best sauce, best people. Go now, thank me later"
3773759,tN1we-fLdL7adeFW6pYYaQ,"Best Mexican in Philly! Everything is fresh and well seasoned. Great Guacamole, tasty beans and rice, amazing tacos. Cactus is the bomb!"
3773760,JKiy0aeyGd3KmXN7uRPFLw,This restaurant is truly amazing!!! The owner is very nice! I've been having trouble looking for a go to taco spot in Philly for a while now but FINALLY I have found my go to taco spot. I'm so thankful for this place. Definitely would recommend it to anyone!!! The presentation is also very nice.
3773761,hpDqORrebPqwvLw06VMGcg,Awesome truck with great food. Staff was nice too! Came to Ridley Fire House and was a popular choice.


# Text Pre-Processing

In [21]:
# Get code to clean text of HTML tags, punctuation?, emoji-like, slang converter (''n'-> ing) or get model that understands yum/yummy
def cleanhtml(x): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', x) 
    return cleantext.replace('\n\n', ' ').replace('\n', '')
    
df_sent2['clean_text'] = df_sent2.apply(lambda row: cleanhtml(row['text']), axis=1)

# Examine sample of dirty comments to see if translation went through
df_sent2

,review_id,text,clean_text
0,BXQcBN0iAi1lAUxibGLFzA,"This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area.","This is nice little Chinese bakery in the heart of Philadelphia's Chinatown! The female cashier was very friendly (flirtatious!) and the pastries shown in nicely adorned display cases. I stopped by early one evening had a sesame ball, which was filled with bean paste. The glutinous rice of the ball was nicely flavored, similar to Bai Tang Gao. Definitely as place worth stopping at if you are in the area."
1,uduvUCvi9w3T2bSGivCfXg,"This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town.","This is the bakery I usually go to in Chinatown. They have a decent variety of buns and pastries. Red bean-filled buns, mung bean-filled buns, coconut stuff, egg tarts etc. If you buy a dozen, they will give you two more for free. The prices are standard for a Chinese bakery, with most things at approximately $1. The service has always been good. They do serve coffee and tea. There are a couple of tables inside, with usually the expected group of older Chinese guys chillin' in the corner table debating about something. Great place to stock up on the buns, which I usually do before leaving town."
2,a0vwPOqDXXZuJkbBW2356g,"A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite.","A delightful find in Chinatown! Very clean, and very kind service. Everyone always seems happy in here, and their treats are absolutely delicious. You can also get a ton of sweet treats without any judgement! They're my favorite."
3,MKNp_CdR2k2202-c8GN5Dw,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future events! Staff was nice also and the cake was really good.,I ordered a graduation cake for my niece and it came out absolutely perfect. Will come again for future events! Staff was nice also and the cake was really good.
4,D1GisLDPe84Rrk_R4X2brQ,"HK-STYLE MILK TEA: FOUR STARS\n\nNot quite sure why there are two Saint Honore Pastries business listings on Yelp, but I have been checking in to this one.\n\nBoyfriend and I go by it all the time while in Chinatown, but it never crossed our mind to try it. I think we might have tried it once a year or two ago, but it wasn't memorable. I decided to try the HK-style milk tea not too long ago and was hooked. I think I went here almost every day one week to get the milk tea, only to realize that I should REALLY watch my sugar intake...it's sooo good! \n\nI personally don't think it's better than Zhong Gang's HK-style milk tea because they have their off days when a certain lady makes them (not enough ice, too much ice, etc.).\n\nOn a side note: I wanted to try the crispy roast pork bun, but accidentally said the wrong name in Chinese and got the roast pork bun. It was decent enough, although I will get to trying the crispy roast pork bun soon! \n\nI will be back~","HK-STYLE MILK TEA: FOUR STARS Not quite sure why there

In [22]:
# need to preprocess text as it was before used to train model (referring to tokenizer_config.json - https://huggingface.co/karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp/blob/main/tokenizer_config.json) 

# Need to lower, split paragraphs to word level, and lemmatize since BERT tokenizer tokenizes on base words so outstandingly becomes outstading + ly which equals two tokens (https://discuss.huggingface.co/t/is-512-token-in-bert-token-or-character-level/16406)
# count tokens
# reform sentences
# take out any columns with more than 512 which is the max by doing a duckdb sql filtering

df_sent2['clean_text'] = df_sent2['clean_text'].str.lower()


# Sentiment Classification

In [23]:
# using Hugging Face sentiment classifier trained on Yelp restaurant reviews

# Set up the inference pipeline using a model from the 🤗 Hub

sentiment_analysis = pipeline(model="karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp")

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [24]:
# test = df_sent2.copy()

# test = (
# test.assign(sentiment = lambda x: x['clean_text'].apply(lambda s: sentiment_analysis(s)))
#     .assign(
#          label = lambda x: x['sentiment'].apply(lambda s: (s[0]['label'])),
#          score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
#     )
# )

# test.head()

In [25]:
# Due to transformer error:
# If you are running a transformer model with HuggingFace, there is a chance that one of the input sentences is longer than 512 tokens. Either truncate or split you sentences. I suspect the shorter sentences are padded to 512 tokens.

#1. get token length
#2. eliminate tokens lengths greater than max/summarize text using chatgpt?